
#1.0 - Tabela Utilizada no Drive

Estou utilizando a ultima tabela disponibilizada pela MEDICINA USP.<br>
Nome da Tabela: NEW-BDIPMama-INTELI(1).csv

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1.1 - Buscando a tabela e importando as bibliotecas utilizadas
Nomeei a tabela como tabela_completa

In [15]:
import pandas as pd
tabela_completa = pd.read_csv ('/content/drive/MyDrive/DRIVE - USPMedicina/NEW-BDIPMamaV11-INTELI (1).csv',delimiter=";")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,24,25,75,76,77,78,89,90,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
tabela_completa['redcap_repeat_instrument'].value_counts()

dados_antropometricos          50137
dados_histopatologicos_mama     4626
registro_de_tumores             4525
Name: redcap_repeat_instrument, dtype: int64

# 2.0 - Observando os dados válidos
Para cada coluna da tabela, consultamos com o value_counts(), afim de consultar a quantidade de dados nas colunas, sendo assim, há a consulta para uma primeira seleção da tabela.<br><br>
Importante destacar que todas as colunas selecionadas até o momento tem mais de 1/3 dos registros válidos.
O restante foi ignorado devido à ausência de registros significativos.

# 3.0 - Premissas assumidas

Assumimos que todos os NaN acima do primeiro dado numérico significa que o dado não foi preenchido no cadastro do paciente, sendo assim, transformaremos este dado nulo em 0.0<br>
Para não perder todos os dados da amostra foi preferível substituir os nulos (Nan) pelo número 0.0 até chegar o primeiro exposto na coluna, este foi replicado na coluna abaixo até encontrar o próximo valor válido, segundo orientações docente para o entendimento dos dados. A partir deste momento, lê-se a tabela da seguinte forma:<br>

*   Caso observe uma coluna onde o primeiro dado é 0.0, significa que este é Nan.

*   Caso encontre uma coluna onde o primeiro dado é 0, este dado é lido como Não.

Optamos por renomear a tabela completa, que está no drive, para tabela tratada, a fim de conseguir fazer consultas na tabela completa ao longo do projeto, sem prejudicar a modelagem dos dados que ocorre simultaneamente.<br><br>

Utilizamos o comando fillna para buscar na coluna um valor válido e repetir nos dados vazios que iriam se repetir normalmente para o mesmo record_id. Posteriormente,a fim de transformar as string dessa coluna em novas colunas com dados numéricos, especificamente, dados de afirmação (Sim) ou negação (Não), utilizamos o método dummies.

Sendo assim, lê-se a tabela da seguinte forma:

*   Caso observe uma coluna onde o primeiro dado é 0.0, significa que este é **Nan**;
*   Caso observe uma coluna onde o primeiro dado é 0, significa que este é lido como **Não**;
*   Caso Observe uma coluna onde o primeiro dado é 1, significa que este é lido como **Sim**;

Fizemos este comando para a coluna "redcap_repeat_instrument" visando replicar a coluna em três dados diferentes, pegando seus respectivos valores e transformando em colunas.

In [16]:
tabela_trat = tabela_completa.fillna(method='ffill').dropna(subset=['redcap_repeat_instrument']) #Utilizei o fillna com o método ffill para buscar o último valor válido e repetir na linha posterior.
tabela_trat['redcap_repeat_instrument'].unique()

tabela_tratada = pd.get_dummies(tabela_trat, columns=['redcap_repeat_instrument', 'antec_fam_cancer_mama']) #Utilizei o método get_dummies na coluna redcap_repeat_instrument para separar o conteúdo dela em novas colunas no final


A saída esperada é a exclusão da coluna "redcap_repeat_instrument", e a criação de três novas colunas, respectivamente:
* "redcap_repeat_instrument_dados_antropometricos";
* "redcap_repeat_instrument_dados_histopatologicos_mama"; e 
* "redcap_repeat_instrument_registro_de_tumores".

Sendo assim, indicando 1 e 0 nos registros do paciente.<br>

Fiz o mesmo processo para a coluna "antec_fam_cancer_mama" que indica se há algum familiar com câncer ou não, buscando transformar as string "Sim" e "Não" em 0 e 1.

Selecionamos as colunas que iremos utilizar, como retiramos as duplicatas com base 

In [17]:
ta_final = tabela_tratada[['record_id', 'ultinfo', 'breast_feeding_time', 'dob','tumor_subtype', 'antec_fam_cancer_mama_Não','antec_fam_cancer_mama_Sim', 'tempo_rep_hormo',
                   'bmi', 'follow_up_days', 'menarche', 'tobaco','alcohol', 'date_last_fu','familial_degree___1','familial_degree___2','familial_degree___3', 'primary_diganosis',
                   'benign','redcap_repeat_instrument_dados_antropometricos']]
tabela_final = ta_final.fillna(0)
tabela_final = tabela_final.drop_duplicates(subset='record_id', keep='last', ignore_index=True)

# 4.0 - Transformando Colunas
Foi necessário transformar a coluna data de nascimento (dob) em idade, visto que será mais fácil saber a idade da paciente quando gerarmos o modelo de predição.<br><br>
A data de nascimento foi calculada até a última consulta do mesmo, e não até a data presente. Preferimos demonstrar desta forma devido a não existência da data de óbito no banco de dados, e também, por nos concentrarmos em encontrar a relação da sobrevida do paciente até a ultima consulta.
<br><br>
Abaixo o código que gera a idade a partir da data de nascimento.

In [25]:
from datetime import datetime, date
tabela_final['dob']
def pegar_idade(nascimento, _date):
  nascimento = datetime.strptime(nascimento,"%d/%m/%Y").date()
  _date = datetime.strptime(_date,"%d/%m/%Y").date()
  idade = _date.year - nascimento.year
  if(nascimento.month == _date.month and nascimento.day >= _date.year):
    idade = idade + 1
  elif(nascimento.month > _date.month):
    idade = idade + 1
  return idade

def convertDate(age):
  return datetime.strptime(age,"%d/%m/%Y").date()
tabela_final['dob'].apply(convertDate)
tabela_final['date_last_fu'].apply(convertDate)
tabela_final['idade'] = ""
for ind in tabela_final.index:
    tabela_final['idade'][ind] = pegar_idade(tabela_final['dob'][ind], tabela_final['date_last_fu'][ind])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
tabela_final

,record_id,ultinfo,breast_feeding_time,dob,tumor_subtype,antec_fam_cancer_mama_Não,antec_fam_cancer_mama_Sim,tempo_rep_hormo,bmi,follow_up_days,...,tobaco,alcohol,date_last_fu,familial_degree___1,familial_degree___2,familial_degree___3,primary_diganosis,benign,redcap_repeat_instrument_dados_antropometricos,idade
0,54,2.0,0.0,28/03/1948,2.0,0,0,0.0,29.0,0.0,...,2.0,2.0,01/01/2021,1.0,0.0,0.0,2.0,2.0,1,74
1,302,4.0,0.0,12/07/1956,3.0,0,0,0.0,25.2,2225.0,...,2.0,2.0,26/04/2014,0.0,0.0,0.0,2.0,2.0,1,59
2,710,2.0,0.0,28/09/1949,2.0,0,0,0.0,25.9,3294.0,...,2.0,2.0,17/11/2016,0.0,0.0,0.0,2.0,2.0,1,67
3,752,2.0,0.0,30/04/1951,3.0,0,0,0.0,35.1,4153.0,...,2.0,2.0,02/05/2019,0.0,0.0,0.0,2.0,2.0,1,68
4,1589,2.0,0.0,09/05/1966,3.0,0,0,0.0,23.2,3290.0,...,2.0,2.0,24/05/2017,0.0,0.0,0.0,2.0,2.0,1,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4129,82057,2.0,84.0,11/06/1959,3.0,0,1,1.0,40.5,441.0,...,2.0,2.0,03/04/2021,0.0,0.0,0.0,2.0,2.0,1,63
4130,82059,2.0,84.0,11/06/1959,3.0,0,1,1.0,28.6,351.0,...,2.0,2.0,22/06/2021,0.0,0.0,0.0,2.0,2.0,1,62
4131,82122,2.0,84.0,11/06/1959,3.0,0,1,1.0,25.4,401.0,...,2.0,2.0,11/08/2021,0.0,0.0,0.0,2.0,2.0,1,62
4132,82205,1.0,84.0,11/06/1959,4.0,0,1,1.0,41.0,337.0,...,2.0,2.0,10/10/2021,0.0,0.0,0.0,2.0,2.0,1,62
